# Probe Request Field Length (per burst)

## Libraries and Configurations

Import configuration files

In [1]:
from configparser import ConfigParser

config = ConfigParser()
config.read("../config.ini")

['../config.ini']

Import **data libraries**

In [2]:
import pandas as pd
import numpy as np

Import **other libraries**

In [3]:
from rich.progress import Progress
from rich import traceback

traceback.install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x10dc8cdd0>>

Custom helper scripts

In [4]:
%cd ..
from scripts import plotHelper, encodingHelper
%cd data_exploration_cleaning

/Users/bacci/Library/CloudStorage/SynologyDrive-giovanni/Research 🌱/Repositories/COMPACT/notebooks
/Users/bacci/Library/CloudStorage/SynologyDrive-giovanni/Research 🌱/Repositories/COMPACT/notebooks/data_exploration_cleaning


## Import Data

In [5]:
# Combined dataframe
combined_df_csv = config["DEFAULT"]["interim_path"] + "combined_df_raw.csv"

In [6]:
df = pd.read_csv(combined_df_csv)
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

/var/folders/kz/mqbx964j4w53vxgzd7rlzx380000gn/T/ipykernel_7807/3875326735.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(combined_df_csv)


In [7]:
to_drop = ["Channel", "DS Channel", "SSID", "Length"]
df = df.drop(to_drop, axis=1)

In [8]:
df

,Timestamp,MAC Address,HT Capabilities,Extended Capabilities,Vendor Specific Tags,Supported Rates,Extended Supported Rates,VHT Capabilities,HE Capabilities,Label
0,2023-05-20 13:52:01.864465952,d2:6b:aa:b5:fb:ed,2d001bff00000000000000000000000000000000000000...,80000000040000020,NaN,82848b96,0c1218243048606c,NaN,23010808180080203002000d009f08000000fdfffdff39...,iPhone12Pro_C
1,2023-05-20 13:52:01.884716034,d2:6b:aa:b5:fb:ed,2d001bff00000000000000000000000000000000000000...,80000000040000020,NaN,82848b96,0c1218243048606c,NaN,23010808180080203002000d009f08000000fdfffdff39...,iPhone12Pro_C
2,2023-05-20 13:52:01.910542011,d2:6b:aa:b5:fb:ed,2d001bff00000000000000000000000000000000000000...,80000000040000020,NaN,82848b96,0c1218243048606c,NaN,23010808180080203002000d009f08000000fdfffdff39...,iPhone12Pro_C
3,2023-05-20 13:52:01.930788994,d2:6b:aa:b5:fb:ed,2d001bff00000000000000000000000000000000000000...,80000000040000020,NaN,82848b96,0c1218243048606c,NaN,23010808180080203002000d009f08000000fdfffdff39...,iPhone12Pro_C
4,2023-05-20 13:52:01.968745947,d2:6b:aa:b5:fb:ed,2d001bff00000000000000000000000000000000000000...,80000000040000020,NaN,82848b96,0c1218243048606c,NaN,23010808180080203002000d009f08000000fdfffdff39...,iPhone12Pro_C
...,...,...,...,...,...,...,...,...,...,...
76699,2021-06-16 12:51:00.708627939,ec:9b:f3:75:8e:40,631117ffff000000000000000000000000000000000000...,88001400040,506f9a1010,02040b16,0c1218243048606c,NaN,NaN,SamsungS6_H
76700,2021-06-16 12:51:00.732495070,ec:9b:f3:75:8e:40,631117ffff000000000000000000000000000000000000...,88001400040,506f9a1010,02040b16,0c1218243048606c,NaN,NaN,SamsungS6_H
76701,2021-06-16 12:51:00.752908945,ec:9b:f3:75:8e:40,631117ffff000000000000000000000000000000000000...,88001400040,506f9a1010,02040b16,0c1218243048606c,NaN,NaN,SamsungS6_H
76702,2021-06-16 12:51:00.776777029,ec:9b:f3:75:8e:40,631117ffff000000000000000000000000000000000000...,88001400040,506f9a1010,02040b16,0c1218243048606c,NaN,NaN,SamsungS6_H


In [9]:
# Convert specified columns to strings
columns_to_convert = [
    "HT Capabilities",
    "Extended Capabilities",
    "Vendor Specific Tags",
    "Supported Rates",
    "Extended Supported Rates",
    "VHT Capabilities",
    "HE Capabilities",
]

for col in columns_to_convert:
    df[col] = df[col].astype(str)


# Define custom aggregation functions for string length
def min_length(series):
    return np.min(series.str.len())


def max_length(series):
    return np.max(series.str.len())


def mean_length(series):
    return np.mean(series.str.len())


# Apply the aggregation
aggregations = {
    col: [min_length, max_length, mean_length] for col in columns_to_convert
}
grouped_df = df.groupby("MAC Address").agg(aggregations)

In [10]:
grouped_df

HT Capabilities                         \
                       min_length max_length mean_length   
MAC Address                                                
00:0f:00:6a:68:8b               3         52   50.310345   
00:be:3b:a2:a8:56              52         52   52.000000   
02:00:00:00:00:00              52         52   52.000000   
02:00:00:00:3e:b2              52         52   52.000000   
02:00:3a:5e:a1:f4              52         52   52.000000   
...                           ...        ...         ...   
fe:f9:ac:47:0d:b7              52         52   52.000000   
fe:f9:fc:fb:83:9e              52         52   52.000000   
fe:fc:07:34:10:69              52         52   52.000000   
fe:fc:aa:d1:89:d1              52         52   52.000000   
fe:ff:ff:0f:64:5d              52         52   52.000000   

                  Extended Capabilities                         \
                             min_length max_length mean_length   
MAC Address                                                      
00:0f:00:6a:68:8b                     3         17   16.517241   
00:be:3b:a2:a8:56                    12         16   12.432268   
02:00:00:00:00:00                    17         17   17.000000   
02:00:00:00:3e:b2                    11         11   11.000000   
02:00:3a:5e:a1:f4                    11         11   11.000000   
...                                 ...        ...         ...   
fe:f9:ac:47:0d:b7                    11         11   11.000000   
fe:f9:fc:fb:83:9e                    15         15   15.000000   
fe:fc:07:34:10:69                    11         11   11.000000   
fe:fc:aa:d1:89:d1                    11         11   11.000000   
fe:ff:ff:0f:64:5d                    11         11   11.000000   

                  Vendor Specific Tags                        Supported Rates  \
                            min_length max_length mean_length      min_length   
MAC Address                                                                     
00:0f:00:6a:68:8b                    3        334  322.586207              16   
00:be:3b:a2:a8:56                   38         38   38.000000               8   
02:00:00:00:00:00                   14         14   14.000000              16   
02:00:00:00:3e:b2                    3          3    3.000000               8   
02:00:3a:5e:a1:f4                    3          3    3.000000               8   
...                                ...        ...         ...             ...   
fe:f9:ac:47:0d:b7                    3          3    3.000000               8   
fe:f9:fc:fb:83:9e                   22         22   22.000000               8   
fe:fc:07:34:10:69                    3          3    3.000000               8   
fe:fc:aa:d1:89:d1                    3          3    3.000000               8   
fe:ff:ff:0f:64:5d                    3          3    3.000000               8   

                   ...             Extended Supported Rates             \
                   ... mean_length               min_length max_length   
MAC Address        ...                                                   
00:0f:00:6a:68:8b  ...        16.0                        3          8   
00:be:3b:a2:a8:56  ...         8.0                       16         16   
02:00:00:00:00:00  ...        16.0                        8          8   
02:00:00:00:3e:b2  ...         8.0                       16         16   
02:00:3a:5e:a1:f4  ...         8.0                       16         16   
...                ...         ...                      ...        ...   
fe:f9:ac:47:0d:b7  ...         8.0                       16         16   
fe:f9:fc:fb:83:9e  ...         8.0                       16         16   
fe:fc:07:34:10:69  ...         8.0                       16         16   
fe:fc:aa:d1:89:d1  ...         8.0                       16         16   
fe:ff:ff:0f:64:5d  ...         8.0                       16         16   

                              VHT Capabilities                         \
       

In [11]:
grouped_df.to_csv(config["DEFAULT"]["reports_path"] + "/CSV/IE_length.csv")